In [16]:
import sys
sys.path.append('..')
from FundamentsStockBrazil.SetorSubsetor import SetorDataScraper
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

codigos_ibovespa = ['ABEV3', 'AZUL4', 'B3SA3']

setor_financeiro = {'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'ITUB4', 'BPAC11', 'ITUB4', 'SANB11', 'IRBR3'}

chrome_driver_path = "/usr/bin/chromedriver"

service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

scraper = SetorDataScraper(setor_financeiro, options, service, acoes=codigos_ibovespa)

dados = scraper.rodar_acoes()


Tempo de execução da ação ABEV3: 0.02 minutos
Tempo de execução da ação AZUL4: 0.02 minutos
Tempo de execução da ação B3SA3: 0.02 minutos


In [17]:
dados

,segmento_listagem,setor,segmento,tic
0,Tradicional - BOVESPA,Consumo não Cíclico,Cervejas e Refrigerantes,ABEV3
1,Nível 2 de Governança Corporativa,Bens Industriais,Transporte Aéreo,AZUL4
2,Novo Mercado,Financeiro,Serviços Financeiros Diversos,B3SA3
